In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
from dateutil.relativedelta import relativedelta
from suntimes import SunTimes  

In [2]:
identifying_columns = ['Stations_id', 'lat', 'lon', 'Management', 'vargroup', 'Referenzjahr']

In [13]:
ds_list = []
for variable_name in ['t2m', 't2min', 't2max', 'vpd', 'tp', 'ssrd']:
    ds_SSA = pd.read_csv(f'C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\results_for_comparing\\Maize_ML_data_SSA_{variable_name}_varieties.csv').dropna(how='all')
    ds_SSA['observed time to beginning of flowering'] = pd.to_timedelta(ds_SSA['observed time to beginning of flowering']).dt.days#, 'D'
    ds_SSA = ds_SSA.loc[ds_SSA['observed time to beginning of flowering'] > 30] #30
    ds_list.append(ds_SSA)

In [14]:
ds_full = ds_list[0]
for variable_ind, variable_name in enumerate(['t2min', 't2max', 'vpd', 'tp', 'ssrd']):
    print(variable_name)
    ds_full = ds_full.merge(ds_list[variable_ind + 1][identifying_columns + [f'{variable_name} at day {n}' for n in range (300)]], on = identifying_columns)

t2min
t2max
vpd
tp
ssrd


In [15]:
ds_full

Unnamed: 0 WC SOS date  Stations_id    lat    lon Management vargroup  \
0             0  2002-12-17            6  -5.60  12.20    Optimal     EIHY   
1             1  2002-12-13            6  -5.60  12.20     Low pH     EPOP   
2             2  2004-02-21            6  -5.60  12.20    Optimal     EIHY   
3             3  2004-02-20            6  -5.60  12.20      Low N     EPOP   
4             4  2004-02-20            6  -5.60  12.20     Low pH     ILPO   
..          ...         ...          ...    ...    ...        ...      ...   
648         650  2002-11-29         1064 -17.67  31.17    Optimal     ILPO   
649         651  2005-12-16         1064 -17.67  31.17    Optimal     EIHY   
650         652  2005-12-16         1064 -17.67  31.17    Optimal     ILHY   
651         653  2002-11-29         1065 -17.70  31.20    Optimal     EIHY   
652         654  2004-12-24         1068 -17.32  31.57    Optimal     EIHY   

     Referenzjahr  t2m at day 0  t2m at day 1  ...  ssrd at day 290  \
0            2003     25.769745     25.896423  ...     9.658886e+06   
1            2003     25.793823     25.691589  ...     8.764601e+06   
2            2004     27.028351     27.053223  ...     8.773044e+06   
3            2004     25.728760     27.028351  ...     9.101346e+06   
4            2004     25.728760     27.028351  ...     9.101346e+06   
..            ...           ...           ...  ...              ...   
648          2003     23.573075     22.922937  ...     7.313598e+06   
649          2006     20.377498     21.426515  ...     1.494711e+07   
650          2006     20.377498     21.426515  ...     1.494711e+07   
651          2003     23.486664     22.826477  ...     7.289511e+06   
652          2005     22.726278     21.564189  ...     1.869859e+07   

     ssrd at day 291  ssrd at day 292  ssrd at day 293  ssrd at day 294  \
0         9659646.00     1.102219e+07      11260762.00      10792349.00   
1         9469954.00     9.761369e+06      10243110.00       9658886.00   
2        11430146.00     1.038891e+07       8619702.00       9001138.00   
3         8773044.00     1.143015e+07      10388910.00       8619702.00   
4         8773044.00     1.143015e+07      10388910.00       8619702.00   
..               ...              ...              ...              ...   
648      16406341.41     1.589115e+07      16649379.69      16669200.35   
649      13952945.40     1.546647e+07      16057893.91      16953820.34   
650      13952945.40     1.546647e+07      16057893.91      16953820.34   
651      16369182.00     1.589359e+07      16648344.00      16660010.00   
652      18989824.92     1.897472e+07      18900802.96      15195503.20   

     ssrd at day 295  ssrd at day 296  ssrd at day 297  ssrd at day 298  \
0         9859685.00       9173226.00       9485542.00      10062592.00   
1         9659646.00      11022186.00      11260762.00      10792349.00   
2         9345962.00      12314494.00      11222416.00      10701659.00   
3         9001138.00       9345962.00      12314494.00      11222416.00   
4         9001138.00       9345962.00      12314494.00      11222416.00   
..               ...              ...              ...              ...   
648      16868633.00      15445393.27      14461798.30      14136792.37   
649      17723068.92      17388141.48      17687864.30      16882917.09   
650      17723068.92      17388141.48      17687864.30      16882917.09   
651      16871210.00      15425299.00      14390605.00      14048230.00   
652      14902014.02      17989384.52      19206937.28      18426382.84   

     ssrd at day 299  
0       8.418770e+06  
1       9.859685e+06  
2       1.062226e+07  
3       1.070166e+07  
4       1.070166e+07  
..               ...  
648     1.252539e+07  
649     1.420593e+07  
650     1.420593e+07  
651     1.259298e+07  
652     1.727084e+07  

[653 rows x 1819 columns]

In [16]:
def get_categories(anthesis_date):
    return np.array([anthesis_date < n for n in range(183)]).squeeze().astype(np.int64)
cats = ds_full[['observed time to beginning of flowering']].apply(get_categories, args = (), axis = 1)
cats_unlisted = cats.apply(lambda x: pd.Series(x)) #to_frame().T#.explode()#.T.explode()
cats_unlisted = cats_unlisted.rename(columns = {n: f'dev stage at day {n}' for n in range(183)})
ds_full = ds_full.merge(cats_unlisted, left_index=True, right_index = True)

In [17]:
ds_full.loc[:, [f'DTF at day {n}' for n in range(193)]] = np.tile(ds_full['observed time to beginning of flowering'].values, (193, 1)).T - np.tile(np.arange(0, 193), (len(ds_full),1))

In [18]:
def get_photoperiods(inputs):
    #inputs = [lon, lat, day of start]
    day_of_start = datetime(inputs.iloc[2].year, inputs.iloc[2].month, inputs.iloc[2].day)
    lon = inputs.iloc[0]
    lat = inputs.iloc[1]
    sun = SunTimes(lon, lat) #can put altitude in also
    lengths = [(pd.to_datetime(sun.setutc(day_of_start + relativedelta(days=n))) - pd.to_datetime(sun.riseutc(day_of_start + relativedelta(days=n)))).seconds/3600 for n in range(170)]
    return np.array(lengths).squeeze().astype(np.float64)

In [19]:
def get_station_locations(dataset, ds_stations):
    ds_stations.index = ds_stations['LocationID']
    lat = [ds_stations._get_value(row, col) for row, col in zip(dataset['sitecode'], ['Latitude' for count in range(len(dataset))])] #station_data.lookup(row_labels = dataset['Stations_id'], col_labels = ['geograph.Breite'])
    lon = [ds_stations._get_value(row, col) for row, col in zip(dataset['sitecode'], ['Longitude' for count in range(len(dataset))])] #station_data._lookup(dataset['Stations_id'], ['geograph.Laenge'])
    dataset['lat'] = lat
    dataset['lon'] = lon
    dataset['lat'] = dataset['lat'].map(lambda x: x[0] if isinstance(x, float) == False else x)
    dataset['lon'] = dataset['lon'].map(lambda x: x[0] if isinstance(x, float) == False else x)
    return dataset

In [10]:
stations_data_lobell = pd.read_csv('C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\African_data\\Lobell2011\\EIL_site_latlon.csv')
ds_full['sitecode'] = ds_full['Stations_id']
ds_full = get_station_locations(ds_full, stations_data_lobell)

C:\Users\wlwc1989\AppData\Local\Temp\ipykernel_23512\3268591058.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ds_full['sitecode'] = ds_full['Stations_id']


In [21]:
ds_full['Planting date'] = pd.to_datetime(ds_full['WC SOS date'])
photoperiods = ds_full[['lon', 'lat', 'Planting date']].apply(get_photoperiods, args = (), axis = 1)
photoperiods_unlisted = photoperiods.apply(lambda x: pd.Series(x)) #to_frame().T#.explode()#.T.explode()
photoperiods_unlisted = photoperiods_unlisted.rename(columns = {n: f'photoperiod at day {n}' for n in range(170)})
ds_full = ds_full.merge(photoperiods_unlisted, left_index=True, right_index = True)

C:\Users\wlwc1989\AppData\Local\Temp\ipykernel_23512\626847059.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ds_full['Planting date'] = pd.to_datetime(ds_full['WC SOS date'])


In [22]:
ds_full.to_csv(f'C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\results_for_comparing\\saved_ML_datasets\\SSA_ML_ERA5_1999_2008_with_varieties.csv')

In [2]:
ds_t2m_DE = pd.read_csv('C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\results_for_comparing\\Maize_ML_data_ERA5_t2m_pd_SOS.csv').dropna()
ds_t2m_DE['observed time to beginning of flowering'] = pd.to_timedelta(ds_t2m_DE['observed time to beginning of flowering']).dt.days#, 'D'
ds_t2m_DE = ds_t2m_DE.loc[ds_t2m_DE['observed time to beginning of flowering'] > 30] #30

In [2]:
ds_list = []
path_home = 'C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\results_for_comparing\\Maize_ML_data_ERA5_'
path_suffix = '_pd_SOS.csv'
path_suffix = '_pd_SOS_90s.csv'
for variable_name in ['t2m', 't2min', 't2max', 'vpd', 'tp', 'ssrd']:
    ds_DE = pd.read_csv(path_home + variable_name + path_suffix).dropna(how='all')
    ds_DE['observed time to beginning of flowering'] = pd.to_timedelta(ds_DE['observed time to beginning of flowering']).dt.days#, 'D'
    ds_DE = ds_DE.loc[ds_DE['observed time to beginning of flowering'] > 30] #30
    ds_list.append(ds_DE)

In [5]:
ds_full = ds_list[0]
for variable_ind, variable_name in enumerate(['t2min', 't2max', 'vpd', 'tp', 'ssrd']):
    ds_full = ds_full.merge(ds_list[variable_ind + 1][identifying_columns + [f'{variable_name} at day {n}' for n in range (310)]], on = identifying_columns)

In [6]:
def get_categories(anthesis_date):
    return np.array([anthesis_date < n for n in range(183)]).squeeze().astype(np.int64)
cats = ds_full[['observed time to beginning of flowering']].apply(get_categories, args = (), axis = 1)
cats_unlisted = cats.apply(lambda x: pd.Series(x)) #to_frame().T#.explode()#.T.explode()
cats_unlisted = cats_unlisted.rename(columns = {n: f'dev stage at day {n}' for n in range(183)})
ds_full = ds_full.merge(cats_unlisted, left_index=True, right_index = True)

In [7]:
ds_full.loc[:, [f'DTF at day {n}' for n in range(193)]] = np.tile(ds_full['observed time to beginning of flowering'].values, (193, 1)).T - np.tile(np.arange(0, 193), (len(ds_full),1))

In [8]:
def get_photoperiods(inputs):
    #inputs = [lon, lat, day of start]
    day_of_start = datetime(inputs.iloc[2].year, inputs.iloc[2].month, inputs.iloc[2].day)
    lon = inputs.iloc[0]
    lat = inputs.iloc[1]
    sun = SunTimes(lon, lat) #can put altitude in also
    lengths = [(pd.to_datetime(sun.setutc(day_of_start + relativedelta(days=n))) - pd.to_datetime(sun.riseutc(day_of_start + relativedelta(days=n)))).seconds/3600 for n in range(170)]
    return np.array(lengths).squeeze().astype(np.float64)

In [9]:
ds_full['Planting date'] = pd.to_datetime(ds_full['Planting date'])
photoperiods = ds_full[['lon', 'lat', 'Planting date']].apply(get_photoperiods, args = (), axis = 1)
photoperiods_unlisted = photoperiods.apply(lambda x: pd.Series(x)) #to_frame().T#.explode()#.T.explode()
photoperiods_unlisted = photoperiods_unlisted.rename(columns = {n: f'photoperiod at day {n}' for n in range(170)})
ds_full = ds_full.merge(photoperiods_unlisted, left_index=True, right_index = True)

In [10]:
ds_full.to_csv(f'C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\results_for_comparing\\saved_ML_datasets\\DE_ML_ERA5_90s.csv')

In [ ]:
ds_full = pd.read_csv(f'C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\results_for_comparing\\saved_ML_datasets\\DE_ML_ERA5_90s.csv')
ds2 = pd.read_csv('C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\results_for_comparing\\saved_ML_datasets\\DE_ML_ERA5_2001_2024.csv')
ds_mega = pd.concat([ds_full, ds2], ignore_index=True, sort=False)
ds_varieties = ds_mega.loc[ds_mega['SORTE'] != ' Mais, Sorte unbekannt'].dropna(subset = ['SORTE'])

In [17]:
ds_mega.loc[ds_mega['SORTE'].isin([' Mais, frühe Reife, FAO 220',
       ' Mais, frühe Reife, FAO 210', ' Mais, mittelfrühe Reife, FAO 240',
       ' Mais, mittelfrühe Reife, FAO 230',
       ' Mais, mittelspäte Reife, FAO 280', ' Mais, frühe Reife, FAO 200',
       ' Mais, frühe Reife, FAO 190',
       ' Mais, mittelfrühe Reife, FAO 250', ' Mais, frühe Reife, FAO 180',
       ' Mais, mittelspäte Reife, FAO 260', ' Mais, späte Reife, FAO 350',
       ' Mais, mittelspäte Reife, FAO 270',
       ' Mais, mittelspäte Reife, FAO 290', ' Mais, späte Reife, FAO 300',
       ' Mais, späte Reife, FAO 310', ' Mais, frühe Reife, FAO 170',
       ' Mais, späte Reife, FAO 320', 
       ' Mais, späte Reife, FAO 330',])]# ==' Mais, frühe Reife, FAO 220']

Unnamed: 0.1  Unnamed: 0 Planting date  Referenzjahr  Stations_id  \
1                 1          10    1991-04-25          1991         7516   
2                 2          11    1992-04-24          1992         7516   
3                 3          12    1993-04-24          1993         7516   
4                 4          13    1994-04-28          1994         7516   
5                 5          14    1995-05-02          1995         7516   
...             ...         ...           ...           ...          ...   
26127         15222       84498    2001-05-06          2001        19734   
26128         15223       84499    2002-04-26          2002        19734   
26155         15250       84600    2001-04-20          2001        19740   
26164         15259       84686    2001-05-03          2001        19741   
26165         15260       84687    2002-05-05          2002        19741   

           lat      lon  Objekt_id Reporting method Historic or recent  ...  \
1      54.4000   9.9667        215           annual           historic  ...   
2      54.4000   9.9667        215           annual           historic  ...   
3      54.4000   9.9667        215           annual           historic  ...   
4      54.4000   9.9667        215           annual           historic  ...   
5      54.4000   9.9667        215           annual           historic  ...   
...        ...      ...        ...              ...                ...  ...   
26127  51.5448  10.8748        215           annual           historic  ...   
26128  51.5448  10.8748        215           annual           historic  ...   
26155  52.8719  13.3927        215           annual           historic  ...   
26164  52.1458  11.9563        215           annual           historic  ...   
26165  52.1458  11.9563        215           annual           historic  ...   

       photoperiod at day 160 photoperiod at day 161  photoperiod at day 162  \
1                   11.533333              11.450000               11.400000   
2                   11.550000              11.483333               11.400000   
3                   11.566667              11.483333               11.433333   
4                   11.300000              11.216667               11.166667   
5                   11.016667              10.950000               10.900000   
...                       ...                    ...                     ...   
26127               10.850000              10.783333               10.716667   
26128               11.516667              11.450000               11.366667   
26155               11.883333              11.800000               11.733333   
26164               11.016667              10.950000               10.883333   
26165               10.900000              10.850000               10.766667   

      photoperiod at day 163  photoperiod at day 164  photoperiod at day 165  \
1                  11.316667               11.250000               11.166667   
2                  11.333333               11.250000               11.183333   
3                  11.350000               11.283333               11.200000   
4                  11.083333               11.016667               10.933333   
5                  10.816667               10.750000               10.666667   
...                      ...                     ...                     ...   
26127              10.650000               10.583333               10.516667   
26128              11.316667               11.250000               11.183333   
26155              11.683333               11.600000               11.533333   
26164              10.816667               10.750000               10.700000   
26165              10.700000               10.633333               10.583333   

       photoperiod at day 166  photoperiod at day 167  photoperiod at day 168  \
1                   11.100000               11.016667               10.950000   
2                   11.116667               11.050000        